# Clip 4

## Cropping

In [4]:
import open3d as o3d
import numpy as np
import json

In [5]:
# Initial
kota = "../data/kota_circuit2.ply"
crop_json = "crop.json"
pcd = o3d.io.read_point_cloud(kota)
print(pcd)
print()

# Load JSON configuration
with open('crop.json', 'r') as f:
    config = json.load(f)

min_bound = config['min_bound']
max_bound = config['max_bound']

print(f"min_bound = {min_bound}")
print(f"max_bound = {max_bound}")

PointCloud with 14640197 points.

min_bound = [-100, -100, -100]
max_bound = [100, 100, 100]


In [6]:
# Define the axis-aligned bounding box with the loaded bounds
aabb = o3d.geometry.AxisAlignedBoundingBox(min_bound=min_bound, max_bound=max_bound)

# Crop the point cloud
cropped_pcd = pcd.crop(aabb)

print(cropped_pcd)

PointCloud with 3240139 points.


## Get vertex normal

In [7]:
# Get vertex normal
cropped_pcd.estimate_normals(
    search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=0.1, max_nn=30))

In [8]:
o3d.io.write_point_cloud("cropped_with_nm.ply", cropped_pcd, write_ascii=True) # save new ascii ply
# o3d.io.write_point_cloud("cropped_with_nm.ply", cropped_pcd) # save new ply

True

In [41]:
# Visualize cropped
o3d.visualization.draw_geometries([cropped_pcd])

[Open3D WARNING] GLFW Error: Cocoa: Failed to find service port for display


2024-03-05 15:13:14.613 Python[33206:1439168] TSM AdjustCapsLockLEDForKeyTransitionHandling - _ISSetPhysicalKeyboardCapsLockLED Inhibit


KeyboardInterrupt: 

## X Y X Filter

In [9]:
# Prepare data for filter

# Convert the point cloud to a NumPy array
points = np.asarray(cropped_pcd.points)
colors = np.asarray(cropped_pcd.colors) 

# Print the min and max of each coordinate
print(f"X range: {points[:, 0].min()} to {points[:, 0].max()}")
print(f"Y range: {points[:, 1].min()} to {points[:, 1].max()}")
print(f"Z range: {points[:, 2].min()} to {points[:, 2].max()}")

X range: -99.99984741210938 to 99.99995422363281
Y range: -99.99994659423828 to 99.9999771118164
Z range: -97.96589660644531 to -45.7867546081543


In [10]:
# filter x y z
# z >= 0.9, -0.2 < x < +0.2, -0.2 < y < +0.2
x_min, x_max = -45, 45
y_min, y_max = -45, 45
z_min, z_max = -70, -65

filtered_indices = (z_min < points[:, 0]) & (points[:, 2] < z_max) & (x_min < points[:, 0]) & (points[:, 0] < x_max) & (y_min < points[:, 1]) & (points[:, 1] < y_max)
filtered_points = points[filtered_indices]
filtered_colors = colors[filtered_indices]  # Apply the same indices to the colors

filtered_pcd = o3d.geometry.PointCloud()
filtered_pcd.points = o3d.utility.Vector3dVector(filtered_points)
filtered_pcd.colors = o3d.utility.Vector3dVector(filtered_colors)  # Set the colors for the filtered points

# Check the number of points in the filtered point cloud
print(f"Number of points after filtering: {len(filtered_points)}")

Number of points after filtering: 595758


In [13]:
# Get vertex normal
filtered_pcd.estimate_normals(
    search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=0.1, max_nn=30))

In [14]:
# Save the filtered point cloud to a new file
o3d.io.write_point_cloud("filtered_point_cloud.ply", filtered_pcd, write_ascii=True)

True

In [15]:
# Visualize the filtered point cloud
o3d.visualization.draw_geometries([filtered_pcd])

[Open3D WARNING] GLFW Error: Cocoa: Failed to find service port for display
[Open3D WARNING] GLFW Error: Cocoa: Failed to find service port for display


2024-03-06 00:16:23.533 Python[39979:1684948] TSM AdjustCapsLockLEDForKeyTransitionHandling - _ISSetPhysicalKeyboardCapsLockLED Inhibit


KeyboardInterrupt: 